<a href="https://colab.research.google.com/github/crychenet/analysis_of_financion_report/blob/master/%D0%9F%D0%BE%D0%B4%D1%81%D0%BE%D1%80%D1%82OZON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import datetime
import re

sales_report = pd.read_excel("/content/2023.10.31 Отчет по скидкам_021123.xlsx", skiprows=1)
initial_df = pd.read_excel("/content/Отчет по товарам за период 2023-10-30 - 2023-11-05.xlsx")
residue_OZON = pd.read_excel("/content/stocks_and_movement_products-report.xlsx")


/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
warehouse_df = pd.read_excel('Склад.xlsx')

In [ ]:
article_df = pd.DataFrame(initial_df['Артикул'].unique())
article_df.rename(columns={0: 'Артикул'}, inplace=True)
article_df = article_df[article_df['Артикул'].notna()]
print(article_df)

                 Артикул
0     19631-9TX-228ШN-39
1     196-60-01(З)N-1-41
2      40MK-82-01A0V4-38
3          88NB-99-04-39
4     19631-9TX-161ШN-38
...                  ...
3252         601-1214-40
3253       1137-491-2-38
3254     150-0201-11Б-36
3255       47MK-98-13-40
3256   41MK-32-01BСCC-40

[3256 rows x 1 columns]


In [ ]:
# warehouse_df = pd.read_excel('Склад.xlsx')
print(f'Начало {datetime.datetime.now()}')


def find_article_and_size(column):
    try:
        first_way = column[:-3]
        second_way = column[:-6]
    except:
        return pd.Series([column, 0])
    test = sales_report[sales_report['Артикул 1С'] == column]
    try:
        if not test.empty:
            return pd.Series([test['Артикул 1С'].iloc[0], 0])
        test = sales_report[sales_report['Артикул 1С'] == first_way]
        if not test.empty:
            return pd.Series([test['Артикул 1С'].iloc[0], column[-2:]])
        test = sales_report[sales_report['Артикул 1С'] == second_way]
        if not test.empty:
            return pd.Series([test['Артикул 1С'].iloc[0], column[-5:]])
    except:
        return pd.Series([column, 0])
    return pd.Series([column, 0])


article_df[['Артикул 1С', 'Размер']] = pd.Series([0, 0])
article_df[['Артикул 1С', 'Размер']] = article_df['Артикул'].apply(find_article_and_size)


def get_number_of_ransoms(column):
    number_of_returns = initial_df[(initial_df['Артикул'] == column) & (
            initial_df['За продажу или возврат до вычета комиссий и услуг'] < 0)]['Количество'].sum()
    number_of_sales = initial_df[(initial_df['Артикул'] == column) & (
            initial_df['За продажу или возврат до вычета комиссий и услуг'] > 0)]['Количество'].sum()
    return number_of_sales - number_of_returns


article_df['Выкуплено'] = np.where(article_df['Артикул'].apply(get_number_of_ransoms) > 0, article_df['Артикул'].apply(get_number_of_ransoms), 0)

Начало 2023-11-09 09:18:56.100705


In [ ]:
print(article_df)

                 Артикул       Артикул 1С Размер  Выкуплено
0     19631-9TX-228ШN-39  19631-9TX-228ШN     39         11
1     196-60-01(З)N-1-41  196-60-01(З)N-1     41         11
2      40MK-82-01A0V4-38   40MK-82-01A0V4     38          0
3          88NB-99-04-39       88NB-99-04     39          3
4     19631-9TX-161ШN-38  19631-9TX-161ШN     38         38
...                  ...              ...    ...        ...
3252         601-1214-40         601-1214     40          0
3253       1137-491-2-38       1137-491-2     38          0
3254     150-0201-11Б-36     150-0201-11Б     36          0
3255       47MK-98-13-40       47MK-98-13     40          0
3256   41MK-32-01BСCC-40   41MK-32-01BСCC     40          0

[3256 rows x 4 columns]


In [ ]:
residue_OZON = residue_OZON[['Артикул', 'Доступный к продаже товар']].groupby('Артикул').sum().reset_index()
article_df = pd.merge(article_df, residue_OZON[['Артикул', 'Доступный к продаже товар']], on='Артикул', how='left')
article_df.rename({'Доступный к продаже товар': 'Текущий остаток (сайт)',
                   'Артикул': 'Артикул (OZON)',
                   'Артикул 1С': 'Артикул'}, inplace=True)
print(article_df)

                 Артикул       Артикул 1С Размер  Выкуплено  \
0     19631-9TX-228ШN-39  19631-9TX-228ШN     39         11   
1     196-60-01(З)N-1-41  196-60-01(З)N-1     41         11   
2      40MK-82-01A0V4-38   40MK-82-01A0V4     38          0   
3          88NB-99-04-39       88NB-99-04     39          3   
4     19631-9TX-161ШN-38  19631-9TX-161ШN     38         38   
...                  ...              ...    ...        ...   
3251         601-1214-40         601-1214     40          0   
3252       1137-491-2-38       1137-491-2     38          0   
3253     150-0201-11Б-36     150-0201-11Б     36          0   
3254       47MK-98-13-40       47MK-98-13     40          0   
3255   41MK-32-01BСCC-40   41MK-32-01BСCC     40          0   

      Доступный к продаже товар  
0                          61.0  
1                           7.0  
2                           6.0  
3                          12.0  
4                          53.0  
...                         ...  
3251 

In [ ]:
article_df.rename(columns={'Доступный к продаже товар': 'Текущий остаток (сайт)',
                   'Артикул': 'Артикул (OZON)',
                   'Артикул 1С': 'Артикул'}, inplace=True)
print(article_df)

          Артикул (OZON)          Артикул Размер  Выкуплено  \
0     19631-9TX-228ШN-39  19631-9TX-228ШN     39         11   
1     196-60-01(З)N-1-41  196-60-01(З)N-1     41         11   
2      40MK-82-01A0V4-38   40MK-82-01A0V4     38          0   
3          88NB-99-04-39       88NB-99-04     39          3   
4     19631-9TX-161ШN-38  19631-9TX-161ШN     38         38   
...                  ...              ...    ...        ...   
3251         601-1214-40         601-1214     40          0   
3252       1137-491-2-38       1137-491-2     38          0   
3253     150-0201-11Б-36     150-0201-11Б     36          0   
3254       47MK-98-13-40       47MK-98-13     40          0   
3255   41MK-32-01BСCC-40   41MK-32-01BСCC     40          0   

      Текущий остаток (сайт)  
0                       61.0  
1                        7.0  
2                        6.0  
3                       12.0  
4                       53.0  
...                      ...  
3251                     N

In [ ]:
def sort_warehouse(pandas_df):
    def calculate_sum(column):
        # Доделать исключения
        try:
            split_column = column.split(' ')
            sum_size_boxe = split_column[1]
            sum_size_boxe = re.sub(r'[(, )]', '', sum_size_boxe)
            split_column[0] = split_column[0].replace(';', ',')
            return split_column[0] + ' ' + str(sum([int(element) for element in list(sum_size_boxe)])) + ' ' + split_column[1]
        except:
            pass # Обратить внимание
    # ТИПА 23/24;25/26;27/28;29/30 (2222) не должно быть!
    pandas_df['Характеристика номенклатуры.Размеры'] = np.where(
        pandas_df['Характеристика номенклатуры.Размеры'].str.contains(';'),
        pandas_df['Характеристика номенклатуры.Размеры'].str.replace(';', ',').apply(calculate_sum),
        pandas_df['Характеристика номенклатуры.Размеры']
    )
    pandas_df['Характеристика номенклатуры.Размеры'] = pandas_df['Характеристика номенклатуры.Размеры'].astype(str)
    # Удалить строку Итог
    article_information = pandas_df['Характеристика номенклатуры.Размеры'].str.split(' ', expand=True)
    article_information.rename(columns={0: 'Размер', 1: 'Кол-во пар', 2: 'Список кол-ва размеров'}, inplace=True)
    # article_information.to_excel('article_information.xlsx')
    sorted_df = pd.DataFrame({
        'Артикул': pandas_df['Артикул'],
        'Размер': article_information['Размер'],
        'Кол-во пар': article_information['Кол-во пар'],
        'Список кол-ва размеров': article_information['Список кол-ва размеров'],
        'Свободно': pandas_df['Свободно'],
        'Способ доставки': np.where(
            (article_information['Список кол-ва размеров'].str.len() == 4) | (article_information['Список кол-ва размеров'].str.len() == 3),
            'Монокороб',
            np.where(
                article_information['Кол-во пар'].isnull(),
                'Штучно',
                'Ростовой короб'
            ))
    })
    return sorted_df


def find_multiplicity_mono_boxe(column):
    find = multiplicity_mono_boxe[multiplicity_mono_boxe['Для поиска'] == column]['Кол-во пар']
    if not find.empty:
        return find.iloc[0]
    else:
        return None


def find_number_mono_boxe(column):
    find = multiplicity_mono_boxe[multiplicity_mono_boxe['Для поиска'] == column]['Свободно']
    if not find.empty:
        return find.iloc[0]
    else:
        return None


def find_multiplicity_rost_boxe(column):
    find = multiplicity_rost_boxe[multiplicity_rost_boxe['Артикул'] == column]['Кол-во пар']
    if not find.empty:
        return find.iloc[0]
    else:
        return None


def find_number_rost_boxe(column):
    find = multiplicity_rost_boxe[multiplicity_rost_boxe['Артикул'] == column]['Свободно']
    if not find.empty:
        return find.iloc[0]
    else:
        return None


def new_need_rost_boxes(column):
    """
    Рассчитывает оптимальное количество ростовых коробов для заказа.

    Функция принимает артикул товара (column) и на основе данных о его продажах и оставшейся потребности
    в товаре рассчитывает оптимальное количество ростовых коробов для заказа.

    :param column: Артикул товара, для которого рассчитывается оптимальное количество ростовых коробов.
    :type column: str
    :return: Оптимальное количество ростовых коробов для заказа.
    :rtype: int
    """
    filtered_df = article_df[(article_df['Артикул'] == column) & pd.to_numeric(
        article_df['Сколько позиций у этого размера в ростовом коробе'], errors='coerce').notna()]
    filtered_df['Оставшаяся потребность'] = filtered_df['Оставшаяся потребность'].astype(float)
    filtered_df['Сколько позиций у этого размера в ростовом коробе'] = filtered_df[
        'Сколько позиций у этого размера в ростовом коробе'].astype(float)

    need_for_size_goods = filtered_df.set_index('Размер')['Оставшаяся потребность'].to_dict()
    goods_distribution = filtered_df.set_index('Размер')['Сколько позиций у этого размера в ростовом коробе'].to_dict()

    if filtered_df.empty:
        return 0
    if all(value == 0 for value in need_for_size_goods.values()):
        return 0
    list_join_penalty = []
    i = 0
    # Цикл выполняет оптимизацию закупки количества ростовых коробов по артикулу
    while True:
        # Расчет производится на основе уменьшения штрафа. Штраф присваивается либо за перегрузку товара на плащадку,
        # либо за недостаточное отправление товара на площадку. Коэфициенты расчета (Перегрузка/Недогрузку = 1/2)
        join_penalty = 0
        if i == 0:
            for key, value in need_for_size_goods.items():
                if value > 0:
                    penalty_size = (value * 2 / 3)
                else:
                    penalty_size = abs(value * 1 / 3)
                join_penalty += penalty_size
            list_join_penalty.append(join_penalty)
        else:
            for key, value in need_for_size_goods.items():
                penalty_size = value - goods_distribution.get(key) * i
                if penalty_size > 0:
                    penalty_size = (penalty_size * 2 / 3)
                else:
                    penalty_size = abs(penalty_size * 1 / 3)
                join_penalty += penalty_size
            list_join_penalty.append(join_penalty)
        try:
            if list_join_penalty[i] > list_join_penalty[i - 1]:
                break
        except IndexError:
            pass
        i += 1
        if i > 20:
            break
    min_index, min_value = min(enumerate(list_join_penalty), key=lambda x: x[1])
    return min_index


# Подсорт производится на основе данных за 1 неделю на 30 дней (Потом завести параметры)
article_df['Хотим поставить'] = article_df['Выкуплено'] / 7 * 30
article_df['Нужно поставить'] = np.where((article_df['Хотим поставить'] - article_df['Текущий остаток (сайт)']) > 0,
                                         article_df['Хотим поставить'] - article_df['Текущий остаток (сайт)'], 0)
# Создаем датафрейм с разобранными данными со склада
sorted_warehouse_df = sort_warehouse(warehouse_df)

multiplicity_mono_boxe = sorted_warehouse_df[sorted_warehouse_df['Монокороб'] == 'Да']
multiplicity_mono_boxe['Для поиска'] = multiplicity_mono_boxe['Артикул'] + multiplicity_mono_boxe['Размер'].astype(str)
multiplicity_rost_boxe = sorted_warehouse_df[sorted_warehouse_df['Монокороб'] == 'Нет']
multiplicity_rost_boxe['Для поиска'] = multiplicity_rost_boxe['Артикул'] + multiplicity_rost_boxe['Размер'].astype(str)

article_df['Для поиска'] = article_df['Артикул'].astype(str) + article_df['Размер'].astype(str)

article_df['Кратность монокороба'] = (article_df['Для поиска']).apply(find_multiplicity_mono_boxe)
article_df['Кол-во товара в мнокоробах'] = (article_df['Для поиска']).apply(find_number_mono_boxe)
article_df.drop(columns=['Для поиска'], inplace=True)

article_df['Кратность рост короба'] = article_df['Артикул'].apply(find_multiplicity_rost_boxe)
article_df['Кол-во товара в рост коробах'] = article_df['Артикул'].apply(find_number_rost_boxe)


dict_size = {}
split_df = sorted_warehouse_df[
    sorted_warehouse_df['Монокороб'] == 'Нет'][['Артикул', 'Размер', 'Список кол-ва размеров']]
for _, row in split_df.iterrows():
    article = row['Артикул']
    size_range = row['Размер']
    size_distribution = row['Список кол-ва размеров']
    values_list = []
    for element in size_distribution[1:-1]:
        values_list.append(int(element))
    if ',' not in size_range:
        keys_list = [i for i in range(int(size_range[0:2]), int(size_range[-2:]) + 1)]
    else:
        size_range = size_range.replace('/', '-')
        keys_list = size_range.split(',')
    dict_size_range = {key: value for key, value in zip(keys_list, values_list)}
    dict_size[article] = dict_size_range


size_df = pd.DataFrame.from_dict(dict_size, orient='index')
number_position_in_rost_box = []

for _, row in article_df.iterrows():
    article = row['Артикул']
    size = row['Размер']
    try:
        number_position_in_rost_box.append(size_df.loc[article, size])
    except KeyError:
        number_position_in_rost_box.append(0)

article_df['Сколько позиций у этого размера в ростовом коробе'] = number_position_in_rost_box

# Получаем данные об отгрузке
dict_count_article = {}
shipment = sales_report[sales_report['Текущая Отгрузка Озон'].notna()][['Артикул 1С', 'Текущая Отгрузка Озон']]
for row in shipment['Артикул 1С']:
    try:
        list_of_possible_types_of_boxes = list(sorted_warehouse_df[sorted_warehouse_df['Артикул'] == row]['Монокороб'])
        dict_count_article[row] = list_of_possible_types_of_boxes
    except IndexError:
        dict_count_article[row] = 'Не найден'
for key, value in dict_count_article.items():
    if len(value) != 1:
        dict_count_article[key] = ['Определить невозможно']
    else:
        dict_count_article[key] = value
df_of_possible_types_of_boxes = pd.DataFrame(dict_count_article).T
df_of_possible_types_of_boxes.reset_index(drop=False, inplace=True)
df_of_possible_types_of_boxes.rename(columns={0: 'Проверка на монокороб', 'index': 'Артикул'}, inplace=True)

article_df = pd.merge(article_df, df_of_possible_types_of_boxes, on='Артикул', how='left')
article_df = pd.merge(article_df, sales_report[['Артикул 1С', 'Текущая Отгрузка Озон']],
                      left_on='Артикул', right_on='Артикул 1С', how='left')
# Создаем данные об информации последней отгрузки
article_df['Отгрузка'] = np.where(article_df['Проверка на монокороб'] == 'Нет',
                                  (article_df['Текущая Отгрузка Озон'].astype(float) /
                                   article_df['Кратность рост короба'].astype(float) *
                                   article_df['Сколько позиций у этого размера в ростовом коробе'].astype(float)), 0)

article_df = article_df.fillna(0)
article_df['Заказываем товара в монокоробах на 30 дней'] = np.floor(np.where(
    np.where(
        article_df['Нужно поставить'] - article_df['Отгрузка'] > article_df['Кол-во товара в мнокоробах'],
        article_df['Кол-во товара в мнокоробах'],
        article_df['Нужно поставить'] - article_df['Отгрузка']) > 0,
    np.where(
        article_df['Нужно поставить'] - article_df['Отгрузка'] > article_df['Кол-во товара в мнокоробах'],
        article_df['Кол-во товара в мнокоробах'],
        article_df['Нужно поставить'] - article_df['Отгрузка']
    ), 0)
)
# Обратить внимаение на around 0.5 = 0
article_df['Заказываем монокоробов'] = np.around(
    (article_df['Заказываем товара в монокоробах на 30 дней'].astype(float) /
     article_df['Кратность монокороба'].astype(float)))

article_df = article_df.fillna(0)
article_df['Оставшаяся потребность'] = np.where(
    article_df['Нужно поставить'] - article_df['Отгрузка'] - article_df['Заказываем товара в монокоробах на 30 дней'] > 0,
    article_df['Нужно поставить'] - article_df['Отгрузка'] - article_df['Заказываем товара в монокоробах на 30 дней'], 0
)

print(f'Начало поиска оптимума по рост коробам {datetime.datetime.now()}')

article_df['Хотим заказать рост коробов на 30 дней'] = article_df['Артикул'].apply(new_need_rost_boxes)
article_df['Заказываем товара в рост коробах на 30 дней'] = np.around(np.where(
    article_df['Хотим заказать рост коробов на 30 дней'] > (article_df['Кол-во товара в рост коробах'] /
                                                            article_df['Кратность рост короба']),
    article_df['Кол-во товара в рост коробах'] / article_df['Кратность рост короба'],
    article_df['Хотим заказать рост коробов на 30 дней'])
)
article_df.to_excel('Подсорт OZON.xlsx')
print(f'Конец {datetime.datetime.now()}')

<ipython-input-45-574a1ce20038>:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  multiplicity_mono_boxe['Для поиска'] = multiplicity_mono_boxe['Артикул'] + multiplicity_mono_boxe['Размер'].astype(str)
<ipython-input-45-574a1ce20038>:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  multiplicity_rost_boxe['Для поиска'] = multiplicity_rost_boxe['Артикул'] + multiplicity_rost_boxe['Размер'].astype(str)


Начало поиска оптимума по рост коробам 2023-11-09 09:33:20.214297


Выходные данные были обрезаны до нескольких последних строк (5000).
See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Оставшаяся потребность'] = filtered_df['Оставшаяся потребность'].astype(float)
<ipython-input-45-574a1ce20038>:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Сколько позиций у этого размера в ростовом коробе'] = filtered_df[
<ipython-input-45-574a1ce20038>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

Конец 2023-11-09 09:33:39.261928
